In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mcap_ros2.reader import read_ros2_messages

%matplotlib qt
# %matplotlib inline

def align_values(data, size):
    selected_indices = np.linspace(0, len(data) - 1, size, dtype=int)
    return data.iloc[selected_indices].reset_index(drop=True)

def quaternion_to_yaw(quaternion):
    q0, q1, q2, q3 = quaternion
    yaw = np.arctan2(2 * (q0 * q3 + q1 * q2), 1 - 2 * (q2**2 + q3**2))
    return yaw

In [9]:
def plot_force(gz_force, python_sim_force):
    fig, ax = plt.subplots(3, 1, sharex=False, sharey=False)
    fig.set_figwidth(32)
    fig.set_figheight(18)
    
    
    ax[0].plot(gz_force[0], label="Gazebo Simulation", color="darkorange")
    ax[0].plot(python_sim_force[0], label="Python Simulation", color="royalblue")
    ax[0].grid()
    ax[0].legend(loc="best")
    
    ax[1].plot(gz_force[1], label="Gazebo Simulation", color="darkorange")
    ax[1].plot(python_sim_force[1], label="Python Simulation", color="royalblue")
    ax[1].grid()
    ax[1].legend(loc="best")
    
    ax[2].plot(gz_force[2], label="Gazebo Simulation", color="darkorange")
    ax[2].plot(python_sim_force[2], label="Python Simulation", color="royalblue")
    ax[2].grid()
    ax[2].legend(loc="best")
    
    ax[0].set_ylabel('X-Axis Force [N]')
    ax[1].set_ylabel('Y-Axis Force [N]')
    ax[2].set_ylabel('Z-Axis Torque [Nm]')
    ax[2].set_xlabel('Time [s]')

In [2]:
gz_hydrodynamics_force = np.empty((3, 0))
gz_hydrodynamics_torque = np.empty((3, 0))
gz_current_force = np.empty((3, 0))
gz_current_torque = np.empty((3, 0))
gz_wind_force = np.empty((3, 0))
gz_wind_torque = np.empty((3, 0))
gz_thrust_vec = np.empty((3, 0))

gz_yaw = []



for msg in read_ros2_messages("../bagfiles/record23dist_p-10_9_90/record23_0.mcap"):
    if msg.channel.topic == "/wave/force":
        gz_hydrodynamics_force = np.column_stack((gz_hydrodynamics_force, np.array([msg.ros_msg.x, msg.ros_msg.y, msg.ros_msg.z])))
        
    if msg.channel.topic == "/wave/torque":
        gz_hydrodynamics_torque = np.column_stack((gz_hydrodynamics_torque, np.array([msg.ros_msg.x, msg.ros_msg.y, msg.ros_msg.z])))
    
    if msg.channel.topic == "/model/vereniki/odometry":
        yaw = quaternion_to_yaw([msg.ros_msg.pose.pose.orientation.w,
                                 msg.ros_msg.pose.pose.orientation.x,
                                 msg.ros_msg.pose.pose.orientation.y,
                                 msg.ros_msg.pose.pose.orientation.z])
        gz_yaw.append(yaw)
    
    if msg.channel.topic == "/waterCurrent/force":
        gz_current_force = np.column_stack((gz_current_force, np.array([msg.ros_msg.x, msg.ros_msg.y, msg.ros_msg.z])))
        
    if msg.channel.topic == "/waterCurrent/torque":
        gz_current_torque = np.column_stack((gz_current_torque, np.array([msg.ros_msg.x, msg.ros_msg.y, msg.ros_msg.z])))
        
    if msg.channel.topic == "/wind/force":
        gz_wind_force = np.column_stack((gz_wind_force, np.array([msg.ros_msg.x, msg.ros_msg.y, msg.ros_msg.z])))
        
    if msg.channel.topic == "/wind/torque":
        gz_wind_torque = np.column_stack((gz_wind_torque, np.array([msg.ros_msg.x, msg.ros_msg.y, msg.ros_msg.z])))
    
    if msg.channel.topic == "/model/vereniki/controller/thrust_vec":
        gz_thrust_vec = np.column_stack((gz_thrust_vec, np.array([msg.ros_msg.x, msg.ros_msg.y, msg.ros_msg.z])))
        

In [3]:
gz_data_size = gz_hydrodynamics_force.shape[1]
print(gz_data_size)

print(len(gz_yaw))
gz_yaw = align_values(pd.DataFrame(gz_yaw), gz_data_size)
print(len(gz_yaw))

732
3050
732


# Hydrodynamics

In [5]:
python_sim = pd.read_csv("simulation_output/hydrodynamics.csv")
python_sim_hydro_force_x = align_values(python_sim.iloc[:, 0], gz_data_size)
python_sim_hydro_force_y = align_values(python_sim.iloc[:, 1], gz_data_size)
python_sim_hydro_torque_z = align_values(python_sim.iloc[:, 2], gz_data_size)

gz_hydrodynamics_force_body = np.zeros((gz_data_size, 2))
i = 0
for force, theta in zip(gz_hydrodynamics_force.T, gz_yaw.values):
    theta = theta[0]
    R = np.array([[np.cos(theta), -np.sin(theta), 0],
                  [np.sin(theta), np.cos(theta), 0],
                  [0, 0, 1]]) 
    rotated_force_vector = np.dot(np.linalg.inv(R), np.array([force[0], force[1], 1]))
    gz_hydrodynamics_force_body[i, 0] = rotated_force_vector[0]
    gz_hydrodynamics_force_body[i, 1] = rotated_force_vector[1]
    i = i + 1

In [6]:
fig, ax = plt.subplots(3, 1, sharex=False, sharey=False)
fig.set_figwidth(32)
fig.set_figheight(18)


ax[0].plot(gz_hydrodynamics_force_body[:, 0], label="Gazebo Simulation", color="darkorange")
ax[0].plot(python_sim_hydro_force_x, label="Python Simulation", color="royalblue")
ax[0].grid()
ax[0].legend(loc="best")

ax[1].plot(gz_hydrodynamics_force_body[:, 1], label="Gazebo Simulation", color="darkorange")
ax[1].plot(python_sim_hydro_force_y, label="Python Simulation", color="royalblue")
ax[1].grid()
ax[1].legend(loc="best")

ax[2].plot(gz_hydrodynamics_torque[2, :], label="Gazebo Simulation", color="darkorange")
ax[2].plot(python_sim_hydro_torque_z, label="Python Simulation", color="royalblue")
ax[2].grid()
ax[2].legend(loc="best")

ax[0].set_ylabel('X-Axis Force [N]')
ax[1].set_ylabel('Y-Axis Force [N]')
ax[2].set_ylabel('Z-Axis Torque [Nm]')
ax[2].set_xlabel('Time [s]')

Text(0.5, 0, 'Time [s]')

In [10]:
plot_force([gz_hydrodynamics_force_body[:, 0], gz_hydrodynamics_force_body[:, 1], gz_hydrodynamics_torque[2, :]], 
           [python_sim_hydro_force_x, python_sim_hydro_force_y, python_sim_hydro_torque_z])

# Thrust

In [7]:
python_sim_thrust = pd.read_csv("simulation_output/thrust.csv")
p